In [2]:
matrix = [[-30,-40,-45,-35,-30],
          [-35,-45,-60,-30,-50],
          [-50,-70,-65,-50,-30],
          [-45,-25,-50,-60,-60]]

In [46]:
cost_matrix = [[30,40,45,35,30],
               [35,45,60,30,50],
               [50,70,65,50,30],
               [45,25,50,60,60]]

In [47]:
#takole se pozene najcenejse prirejanje
hungarian = Hungarian(matrix)
hungarian.calculate()

In [48]:
#takole pa najdrazje (najvecji dobicek)
hungarian = Hungarian(cost_matrix, is_profit_matrix=True)
hungarian.calculate()

In [49]:
result = hungarian.get_results()
potential = hungarian.get_total_potential()

In [50]:
result

[(2, 1), (0, 2), (1, 4), (3, 3)]

In [51]:
potential

225

# Megaron

In [225]:
#izberi dan, od kdaj do kdaj, dobi datume,dobi datume

In [226]:
import time
import datetime
import pandas as pd
import numpy as np
from datetime import timezone
import csv
from hungarian import Hungarian

In [227]:
def min_max_timestamp(leto):
    min_timestamp = datetime.datetime(leto,1,1).replace(tzinfo=timezone.utc).timestamp()
    max_timestamp = datetime.datetime(leto,12,31).replace(tzinfo=timezone.utc).timestamp()
    return (min_timestamp,max_timestamp)

def min_max_timestamp_schoolyear(leto_zacetek):
    '''nekako studentsko solsko leto'''
    min_timestamp = (datetime.datetime(leto_zacetek,10,1).replace(tzinfo=timezone.utc).timestamp())
    max_timestamp = (datetime.datetime(leto_zacetek+1,6,25).replace(tzinfo=timezone.utc).timestamp())
    return (min_timestamp,max_timestamp)

def gen_days(min_timestamp, max_timestamp, iso_weekday):
    '''creates a list of every day between timestamps, that is the weekday (1 monday, 7 sunday)'''
    
    day = 60*60*24
    week = day*7
    
    min_weekday = datetime.datetime.fromtimestamp(min_timestamp).replace(tzinfo=timezone.utc).isoweekday()
    diff = iso_weekday-min_weekday
    if (diff < 0):
        diff += 7
    
    timestamp_list = []
    timestamp  = min_timestamp + diff*day
    while(timestamp <= max_timestamp):
        timestamp_list.append(timestamp)
        timestamp += week
    return timestamp_list

In [284]:
a = min_max_timestamp_schoolyear(2018)
vaje = [int(x) for x in gen_days(a[0],a[1],2)]
#vaje_pandas = pd.to_datetime(vaje, unit='s')

In [295]:
#preberemo rojstne dneve
df = pd.read_csv('lanski.csv', names = ['name','date'])
leto = datetime.datetime.now().year
df['day'] = df['date'].astype(str).str[:2]
df['month'] = df['date'].astype(str).str[3:5]
df['year'] = np.where((df['month'].astype(int) < 8),leto+1,leto)
df['timestamp'] = pd.to_datetime(df[['year', 'month', 'day']])
ljudje = df[['name', 'timestamp']].copy()

In [296]:
imena = ljudje['name'].values.tolist()
rojstni_dnevi = [ int(x / 1000000000) for x in ljudje['timestamp'].values.tolist()]

In [297]:
#ker castita dva na vajo podvojimo vsak element
vaje_vozlisca = [x for x in vaje for _ in (0, 1)]

In [298]:
#vsak stolpec ena vaja, vrstica en clovek
matrika = [[int(abs(vaja-clovek)/86400) for vaja in vaje_vozlisca] for clovek in rojstni_dnevi]

In [299]:
matching = Hungarian(matrika)
matching.calculate()

In [300]:
result = matching.get_results()
potential = matching.get_total_potential()

In [301]:
result = sorted(result,key=lambda tup: tup[1])

In [302]:
with open('razpored.csv', 'w') as csvfile:
    fieldnames = ['vaja', 'ime','rojstni_dan','dni']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,delimiter = ';')

    writer.writeheader()
    for par in result:
        ime = imena[par[0]]
        vaja = datetime.datetime.fromtimestamp(vaje_vozlisca[par[1]]).replace(tzinfo=timezone.utc).date()
        rojstni_dan = datetime.datetime.fromtimestamp(rojstni_dnevi[par[0]]).replace(tzinfo=timezone.utc).date()
        razdalja = abs(rojstni_dan-vaja).days
        #print(ime,str(vaja),str(rojstni_dan),razdalja)
        writer.writerow({'vaja': vaja, 'ime': ime, 'rojstni_dan' : rojstni_dan, 'dni': razdalja})

In [303]:
potential

1224